# 药物分子-溶剂水交换结合自由能分子动力学模拟

https://github.com/quantaosun  


这是一个用来计算一个蛋白质靶点和一个小分子药物之间结合自由能的笔记本，使用的是英国的开源软件Sire的一个模块，名字为Waterswap，如果你希望理解背后的逻辑，你需要仔细阅读下面的两篇文献。

你也许没有很好的个人笔记本，就像我一样。但你仍然可以尝试在谷歌上的免费算力平台collab上使用，注意及时保存到你的网盘里。

缺点是这个笔记本不支持GPU加速。

好消息是，即使你使用个人笔记本或者配置较低的服务器，你可以把你的任务无限次的延续，只要你有耐心，总会完成。而计算任务会自动从上次你结束的地方继续计算，你无需做任何其他设置。

如果你有许多CPU比如超过50块，那么你的计算可能会在一到两天内完成。

Woods, C. J., Malaisree, M., Hannongbua, S., Mulholland, A.J., “A water-swap reaction coordinate for the calculation of absolute protein-ligand binding free energies”, J. Chem. Phys. 134, 054114, 2011, DOI:10.1063/1.3519057 

Woods, C. J., Malaisree, M., Michel, J., Long, B., McIntosh-Smith, S., Mulholland, A. J., “Rapid Decomposition and Visualisation of Protein-Ligand Binding Free Energies by Residue and by Water”, Faraday Discussions 169: Molecular Simulation and Visualisation, 2014, DOI:10.1039/C3FD00125C



请注意，如果你是在你自己笔记本上或者自己的服务器上运行，你无需运行任何于谷歌网盘相关的命令。

方法点评： 这个方法根据个人的使用感受（github/quantaosun），原理上不如FEP，但是要好于MMPBSA。但FEP对硬件的苛刻要求和繁琐的使用技巧，后者昂贵的价格，往往让人望而却步。

这个方法是完全免费的哦，唯一的硬伤也许就是速度慢一些，并且跟MMPBSA一样结果的数值区间是不能落入真实的实验区间的，但两个化合物直接的差值是具有可比性的。

In [1]:
#@title ❤Install Omnia
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh 
! chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh 
! bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh  -b -f -p /usr/local 
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
#安装sire
!yes | conda install -c conda-forge -c omnia -c michellab sire

--2022-11-16 21:43:47--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104996770 (100M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh’

Miniconda3-py37_4.1 100%[===================>] 100.13M  96.7MB/s    in 1.0s    

2022-11-16 21:43:48 (96.7 MB/s) - ‘Miniconda3-py37_4.12.0-Linux-x86_64.sh’ saved [104996770/104996770]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.15.0=py37hd667e15_1
   

In [ ]:
#@title ❤ABF waterswap validation
!optimise_openmm
#现在我们就能直接使用waterswap 了，我们将使用这个工具进行相对结合自由能的计算
!waterswap -h

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! grep -v %COMMENT /content/SYS_gaff2.prmtop > SYS.prmtop

In [ ]:
#@title Copy this every less than 24 hours back to Drive, then copy back and restart!
#!cp -r /content/output /content/drive/MyDrive/waterswap
#!cp *s3* /content/drive/MyDrive/waterswap
#!cp *wsrc* /content/drive/MyDrive/waterswap
#!cp *swap* /content/drive/MyDrive/waterswap
#!cp *SYS* /content/drive/MyDrive/waterswap

In [ ]:
#@title Copy back all intermediate restart files back to colab, and change the -n steps, to restart the simulation. This  is COOL!
#!cp -r /content/drive/MyDrive/waterswap /content


下面这个例子是让第一次计算跑到500步骤，但即使因为这样那样的原因没有完成，你也完全不必担心，加入任务跑到了250步骤，则下次重复运行时，会在动从251步开始，你也许都察觉不到这背后跟第一次运行时的区别。

In [ ]:
%cd waterswap
!waterswap -l LIG  -t SYS.prmtop -c SYS_gaff2.crd -n 500

[Errno 2] No such file or directory: 'waterswap'
/content
Starting waterswap: number of threads equals 2


Running a waterswap calculation using files SYS.prmtop and SYS_gaff2.crd.
The absolute binding free energy of the molecule containing residue LIG will be calculated.

Number of iterations to perform == 500

Using parameters:
ligand name == LIG
nmoves == 500
protein crdfile == SYS_gaff2.crd
protein topfile == SYS.prmtop
Loading the protein box system...
Loading from Amber files SYS.prmtop / SYS_gaff2.crd...

Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.

Loading the molecules from the files "SYS_gaff2.crd" and "SYS.prmtop"...
Number of molecules == 17650
System space == P

In [ ]:
!cp -r /content/waterswap /content/drive/MyDrive

请至少跑到1000步，再运行分析指令。具体的原理请参考以上两篇文献。

In [ ]:
#Analysi
!analyse_freenrg -i output/freenrgs.s3 -o results.txt

误差评估： 结果会给出三个结果，如果你的模拟比较好，这三个数值应当接近。